# Module 1 Partie 2

### Création d'un espace de travail Azure

Un espace de travail est un ensemble de ressources lui même contenu dans un groupe de ressources:

Mircosoft ==>  Abonnement ==>  Groupe de ressources: 
                                        - Compte de stockage
                                        - application insights
                                        - coffre clés
                                        - VM
                                        - Regisre de conteneurs
                                        - **Espace de travail** 
                                                    |
                                                    |==>  - calcul 
                                                          - Notebook
                                                          - Pipeline
                                                          - Données
                                                          - Expérience
                                                          - Modèles


**Espace de travail**

calcul: cible de calcul, d'entrainement + deploiement 
Notebook
Pipeline: définit porcessus orchestré en plusieurs étapes
Données: pour experimentation + entrainement
Expérience: + historique execution ac métriques+sorties journalières                                                 Modèles (entrainés)

**groupe de ressources**

Compte de sotckage: stocker fichiers + données d'entraiement utilisé dans espace de travail (ET)
Instance Application insights: superviser services prédictif de l'ET
Instance Azure key Vault :  gestion éléments secret tq les clés d'auth.
VM
Registre de conteneurs : gestion conteneurs pour modèles déployés 

**3 manières** de créer un espace de travail:

1- via le portail (clique bouton)

2- via un SDK python

In [5]:
from azureml.core import Workspace
    
ws = Workspace.create(name='aml-workspace', 
                      subscription_id='6f1ad109-aa25-...',
                      resource_group='aml-resourcesgroup',
                      create_resource_group=True,
                      location='eastus',
                      sku='enterprise'
                     )

Deploying StorageAccount with name amlworksstorage03dda8055.
Deploying AppInsights with name amlworksinsights28d1c4e3.
Deployed AppInsights with name amlworksinsights28d1c4e3. Took 18.91 seconds.
Deploying KeyVault with name amlworkskeyvaultde88dfd7.
Deployed KeyVault with name amlworkskeyvaultde88dfd7. Took 36.91 seconds.
Deployed StorageAccount with name amlworksstorage03dda8055. Took 40.89 seconds.
Deploying Workspace with name aml-workspace.
Deployed Workspace with name aml-workspace. Took 117.63 seconds.


3- via l’interface de ligne de commande (CLI) Azure avec l’extension CLI Azure Machine Learning

In [ ]:
az ml workspace create -w 'aml-workspace' -g 'aml-resources'


In [ ]:
#!pip install azureml-sdk

Le SDK comprend des bonus facultatifs qui ne sont pas nécessaires pour les opérations de base. Exemple, 
- le bonus **notebooks** inclut des widgets pour afficher une sortie détaillée dans les notebooks Jupyter, 
- le bonus **automl** inclut des packages pour l’entraînement de machine learning automatisé 
- le bonus **explain** inclut des packages pour la génération d’explications de modèle

In [ ]:
#installation de ces bonus 
!pip install azureml-sdk[notebooks,automl,explain]

Pour se connecter à un espace de travail --> utilisation d'un fichier de configuration d’espace de travail: fichier json qui comprends les détails de l'abonnement azure du type:

{
    **"subscription_id"**: "1234567-abcde-890-fgh...",
    **"resource_group"**: "aml-resources",
    **"workspace_name"**: "aml-workspace"
}

par exemple nous on a fait

{
    "subscription_id": "6f1ad109-aa25-4669-a078-5ccf43db778e",
    "resource_group": "aml-ressourcesgroup",
    "workspace_name": "aml-workspace"
}

In [3]:
from azureml.core import Workspace

ws = Workspace.from_config("/Users/louiserodriguez/Downloads/config.json")

WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.


Par défaut, la méthode from_config recherche un fichier nommé **config.json** dans le dossier qui contient le fichier de code Python, mais vous pouvez spécifier un autre chemin si nécessaire.

Et sinon, autre alternative:

In [ ]:
from azureml.core import Workspace

ws = Workspace.get(name='aml-workspace',
                   subscription_id='1234567-abcde-890-fgh...',
                   resource_group='aml-resources')

In [ ]:
Rmq: authentification nécessaire quelque soit la technique

**Elements d'info à remplir**:

1) le nom de la ressource

2) le nom du groupe de ressources

3) l'identifiant (subscription id)

4) la location/region

5) l'edition de l'espace de travail: ENTREPRISE/BASE

### Extension CLI azure ML

az extension add -n azure-cli-ml

-g : raccurcis pour ressources groupe
-w : raccourcis pour workspace name

### Instance de calcul (VM)

Fournit un environnement de dvp géré avec les autres ressources de l'ET

### Visual Studio Code

Package Azure ML : fournit interface graphique pour l"utilisation des ressoucres dns un espace de tavail Azure

### Les Experiences chez Azure ML

Une experience : c'est un processus nommé, cela peut être l'execution d'un script ou pipeline qui peut généer des métriques et sorties et faire objet d'un suivit dans l'espace de travail

Peut être executé plusieurs fois avec des données, script et paramètres différents. 

In [ ]:
from azureml.core import Experiment

# create an experiment variable
experiment = Experiment(workspace = ws, name = "my-experiment")

# start the experiment
run = experiment.start_logging()

# experiment code goes here

# end the experiment
run.complete()

Une fois l’exécution de l’expérience terminée, vous pouvez voir ses détails sous l’onglet Expériences dans Azure Machine Learning Studio.

##### Journalisation de métrique et création de sorties

Experiences génèrent des **fichiers journaux**:
- possibilité d'ajout de print pour écrire message dans journal
- possibilité d'utiliser l'objet Run et ses fonctions  pour enregistrer 1 valeure nommées, une list une ligne avec plusieurs colonnes, 1 dictionnaire, 1 fichier image



Exemple, le code suivant enregistre le nombre d’observations (enregistrements) dans un fichier CSV

In [ ]:
from azureml.core import Experiment
import pandas as pd

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = 'my-experiment')

# Start logging data from the experiment
run = experiment.start_logging()

# load the dataset and count the rows
data = pd.read_csv('data.csv')
row_count = (len(data))

# Log the row count: run.log enregistre 1 ligne avec plusieurs colonnes
run.log('observations', row_count)

# Complete the experiment
run.complete()

##### Recupération et affichage des métriques journalisées 

Première façon:

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

#run étant l'experience créée précedemment 

Deuxième façon:

In [ ]:
#obtnenir le résultat au format json en sortie du type
'''
{
  "observations": 15000
}
'''
import json

# Get logged metrics
metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

##### Fichiers de sortie d’une expérience


Charger des fichiers locaux dans le dossier output de l’exécution à l’aide de la méthode upload_file de l’objet Run 

In [ ]:
run.upload_file(name='outputs/sample.csv', path_or_stream='./sample.csv')

Tous les fichiers écrits dans le dossier de sorties dans le contexte de calcul sont automatiquement chargés sur le dossier de sorties.

Récupérer une liste de fichiers de sortie à partir de l’objet Run 

In [ ]:
import json

files = run.get_file_names()
print(json.dumps(files, indent=2))

### Execution d'un script en tant qu'experience

pouvez exécuter une expérience inline à l’aide de la méthode **start_logging** de l’objet Experiment, mais il est plus courant d’encapsuler la logique d’expérience dans un script et d’exécuter le script en tant qu’expérience.

Un script d’expérience = un fichier de code Python qui contient le code que vous souhaitez exécuter dans l’expérience


Pour accéder au contexte d’exécution de l’expérience (qui est nécessaire pour journaliser les métriques), le script doit importer la classe **azureml.core.Run** et appeler sa méthode **get_context**. Le script peut ensuite utiliser le contexte d’exécution pour journaliser les métriques, charger des fichiers et effectuer l’expérience

In [ ]:
from azureml.core import Run
import pandas as pd
import matplotlib.pyplot as plt
import os

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
data = pd.read_csv('data.csv')

# Count the rows and log the result
row_count = (len(data))
run.log('observations', row_count)

# Save a sample of the data
os.makedirs('outputs', exist_ok=True)
data.sample(100).to_csv("outputs/sample.csv", index=False, header=True)

# Complete the run
run.complete()

RunConfiguration () : configuration d’exécution qui définit l’environnement Python pour l’expérience
ScriptRunConfig() : configuration d’exécution de script qui associe l’environnement d’exécution au script.

**pas trop compris .. à revoir  Q: equivalent à se mettre sur les mêmes bases (données, noyau etc) avant de lancer un potentiel nouveau script ???)**

In [ ]:
from azureml.core import Experiment, RunConfiguration, ScriptRunConfig

# create a new RunConfig object
experiment_run_config = RunConfiguration()

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder, 
                      script='experiment.py',
                      run_config=experiment_run_config) 

# submit the experiment
experiment = Experiment(workspace = ws, name = 'my-experiment')
run = experiment.submit(config=script_config)
run.wait_for_completion(show_output=True)

#### Exercices

Note that the RunDetails widget includes a link to view the run in Azure Machine Learning studio. Click this to open a new browser tab with the run details (you can also just open Azure Machine Learning studio and find the run on the Experiments page). When viewing the run in Azure Machine Learning studio, note the following:

The Properties tab contains the general properties of the experiment run.
The Metrics tab enables you to select logged metrics and view them as tables or charts.
The Images tab enables you to select and view any images or plots that were logged in the experiment (in this case, the Label Distribution plot)
The Child Runs tab lists any child runs (in this experiment there are none).
The Outputs tab shows the output files generated by the experiment.
The Logs tab shows any logs that were generated by the compute context for the experiment (in this case, the experiment was run inline so there are no logs).
The Snapshots tab contains all files in the folder where the experiment code was run (in this case, everything in the same folder as this notebook).
The Raw JSON tab shows a JSON representation of the experiment details.
The Explanations tab is used to show model explanations generated by the experiment (in this case, there are none).

1. Create a *Run Configuration* that defines the Python code execution environment for the script - in this case, it will automatically create a Conda environment with some default Python packages installed.
2. Create a *Script Configuration* that identifies the Python script file to be run in the experiment, and the environment in which to run it

In [ ]:
%%writefile $folder_name/diabetes_experiment.py
from azureml.core import Run
import pandas as pd
import os

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
data = pd.read_csv('diabetes.csv')

# Count the rows and log the result
row_count = (len(data))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# Count and log the label counts
diabetic_counts = data['Diabetic'].value_counts()
print(diabetic_counts)
for k, v in diabetic_counts.items():
    run.log('Label:' + str(k), v)
      
# Save a sample of the data in the outputs folder (which gets uploaded automatically)
os.makedirs('outputs', exist_ok=True)
data.sample(100).to_csv("outputs/sample.csv", index=False, header=True)

# Complete the run
run.complete()

In [7]:
import os
import sys
from azureml.core import Experiment, RunConfiguration, ScriptRunConfig
from azureml.widgets import RunDetails

# create a new RunConfig object
experiment_run_config = RunConfiguration()

# Create a script config
src = ScriptRunConfig(source_directory=experiment_folder, #non du fichier où se trouve le fichier python
                      script='diabetes_experiment.py',   #script python à lancer 
                      run_config=experiment_run_config) 

# submit the experiment
experiment = Experiment(workspace = ws, name = 'diabetes-experiment')
run = experiment.submit(config=src)
RunDetails(run).show()
run.wait_for_completion()

NameError: name 'experiment_folder' is not defined

View experiment run history

In [ ]:
from azureml.core import Experiment, Run

diabetes_experiment = ws.experiments['diabetes-experiment']
for logged_run in diabetes_experiment.get_runs():
    print('Run ID:', logged_run.id)
    metrics = logged_run.get_metrics()
    for key in metrics.keys():
        print('-', key, metrics.get(key))

A retenir en mots clés:

**création de l'espace de travail**

 <font color='green'>ws</font> = Workspace.get(name='aml-workspace',
                                       subscription_id='1234567-abcde-890-fgh...',
                                       resource_group='aml-resources')

   
**création script python  <font color='blue'>fichier.py</font> : Get the experiment run context**
                    
                    
run = Run.get_context()
|
code py + run.log 
|
run.complete()



**faire appel au script préalablement crée pour le lancer: Create a new RunConfig object:**

<font color='red'>experiment_run_config</font> = RunConfiguration()

==> Définit l'environnement d'execution python pour le script. Il crée automatiquement un environnement conda avec packages.**



**Create a script config:**

src = ScriptRunConfig(source_directory=experiment_folder, 
                      script= <font color='blue'>'fichier.py'</font>,
                      run_config=<font color='red'>experiment_run_config</font>)
    
==> Créer une config script qui identifie le fichier ython à lancer dans l'experience et l'environnemnt dans lequel le lancer**





**Submit the experiment:**


experiment = Experiment(workspace = <font color='green'>ws</font>, name = 'diabetes-experiment')
run = experiment.submit(config=src)
RunDetails(run).show() 

==> execution de l'experience**